In [4]:
from scipy import optimize
import numpy as np
from tolsolvty import tolsolvty
import plotly.graph_objects as go

In [5]:
def _calcfg(infA, supA, infb, supb, x):
        #
        #   функция, которая вычисляет значение f максимизируемого распознающего
        #   функционала и его суперградиент g;  кроме того, она выдаёт вектор tt
        #   из значений образующих функционала в данной точке аргумента
        #
        Ac = 0.5 * (infA + supA)
        Ar = 0.5 * (supA - infA)
        bc = 0.5 * (infb + supb)
        br = 0.5 * (supb - infb)
        weight = np.ones((np.size(Ac,0), 1))
        absx = abs(x)

        Acx = Ac @ x
        Arabsx = Ar @ absx

        infs = bc - (Acx - Arabsx)
        sups = bc - (Acx + Arabsx)

        tt = weight * (br - np.maximum(abs(infs), abs(sups)))
    
        mc = np.argmin(tt)
        f = tt[mc]
        infAmc = Ac[[mc], :].T
        supAmc = Ac[[mc], :].T

        xneg = x < 0
        xnonneg = x >= 0

        dl = infAmc * xneg + supAmc * xnonneg
        ds = supAmc * xneg + infAmc * xnonneg
        if np.any(-infs[mc] <= sups[mc]):
            g = weight[mc] * ds
        else:
            g = -weight[mc] * dl

        return [f, g, tt]

In [15]:
def draw_tol_plot(A, b_min, b_max):
    #  Функция для изображения графика распознающего функционала
    #  использует функцию calcfg из tolsolvty для подсчета значения
    x = np.arange(-1.2, -0.6, 0.001)
    y = np.arange(-2.45, -2.35, 0.001)
    z = np.zeros((x.shape[0], y.shape[0]))
    for i in range(x.shape[0]):
        for j in range(y.shape[0]):
            vec = np.array([[0.45295137],[x[i]], [y[j]]])
            z[i, j] = _calcfg(A, A, b_min, b_max, vec)[0][0]

    #fig = plt.figure()
    #ax = fig.add_subplot(111, projection='3d')
    #ax.view_init(30, 100)
    #layout = go.Layout(title="Tol(X2,X3)", xaxis_title="X2", yaxis_title="X3")
    fig = go.Figure(data=[go.Surface(z=z, x=x, y=y)])
    # ax.plot([0], [-0.06], [z[100, 94]], markerfacecolor='c', markeredgecolor='c', marker='*', markersize=10)
    #print(z[100, 94])
    

    fig.show()

In [7]:
c = np.array([0, 0, 0, 1, 1, 1])
A = np.array([[20, 1.2, 1], [0, -1, -2], [0, 1, 2]])
n = np.size(A, 0)
b_min = np.array([4, 7, -5]).reshape(n, 1)
b_max = np.array([6, 9, -3]).reshape(n, 1)
#A = np.array([[1, 2, 3], [1.1, 2, 3],[10, 0, 0]])
#n = np.size(A, 0)
#b_min = np.array([1, 7, -1]).reshape(n, 1)
#b_max = np.array([2, 8, 1]).reshape(n, 1)
rad_b = 0.5 * (b_max-b_min) 
mid_b = 0.5 * (b_max+b_min)
bounds = np.array([(None, None), (None, None), (None, None), (0, None), (0, None), (0, None)])
bounds_pos = np.array([(0.45295137, 0.45295137), (None, None), (None, None), (0, None), (0, None), (0, None)])

In [8]:
C = np.hstack((np.vstack((-A, A)), np.vstack((np.diag(-rad_b.reshape(1, n)[0]), np.diag(-rad_b.reshape(1,n)[0])))))
b_ = np.concatenate((-mid_b.reshape(1, n)[0], mid_b.reshape(1,n)[0])) 
b_

array([-5., -8.,  4.,  5.,  8., -4.])

In [9]:
res = optimize.linprog(c, C, b_, bounds=bounds, method='interior-point')
x, w = np.split(res.x, 2)
print('x=', x)
print('w=', w)
#x = np.array([0.45295137], [-1.18731733], [-2.40634126])

x= [ 0.41928894 -0.66895173 -2.58303679]
w= [3.46249942e-10 2.16497469e+00 1.83502531e+00]


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray



In [10]:
x_min = _calcfg(A, A, b_min, b_max, x.reshape(n,1))[0]
x_min

array([-1.16497469])

In [11]:
[tolmax, argmax, envs, ccode] = tolsolvty(A, A, b_min, b_max)
print('maxTol=', tolmax)
print('argmaxTol=', argmax)

 
 Допусковое множество решений интервальной линейной системы пусто 
 
maxTol= [-1.00000015]
argmaxTol= [[ 0.45295137]
 [-1.18731733]
 [-2.40634126]]


In [16]:
draw_tol_plot(A, b_min, b_max)